# W205 Final Project

Load data from BigQuery

In [1]:
import apiclient

In [2]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

from sklearn.cross_validation import train_test_split


In [3]:
filename = open("My First Project-1f86b13aa6c6.json")
key = filename.read()

#Load BigQuery data
power = pd.read_gbq('SELECT x.*, y.sales FROM heatwave.power_price as x join heatwave.power_sales as y on y.state = x.state and y.sector = x.sector and x.year = y.year and x.month = y.month', 
                    project_id = 'crested-trilogy-136023',
                    private_key = key)

outage = pd.read_gbq('SELECT * from heatwave.power_outage_v2', 
                    project_id = 'crested-trilogy-136023',
                    private_key = key)

weather = pd.read_gbq('SELECT * from heatwave.weather_state', 
                    project_id = 'crested-trilogy-136023',
                    private_key = key)



Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 1; 100.0% done. Elapsed 7.75 s.
Got 34040 rows.

Total time taken 8.86 s.
Finished at 2016-08-15 14:59:39.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 1955 rows.

Total time taken 2.59 s.
Finished at 2016-08-15 14:59:42.
Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 8975 rows.

Total time taken 2.75 s.
Finished at 2016-08-15 14:59:45.


In [4]:
p_gen = pd.read_gbq('SELECT * from heatwave.power_gen', 
                    project_id = 'crested-trilogy-136023',
                    private_key = key)

power_agg = power.groupby(['x_STATE','x_YEAR','x_MONTH'])['y_sales'].sum()
p_gen = p_gen.sort(['STATE', 'YEAR','MONTH'], ascending=[1,1,1])

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 9384 rows.

Total time taken 2.83 s.
Finished at 2016-08-15 14:59:48.


C:\Users\mmorenocabrera\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Merge data sets

In [5]:
result = weather.sort(['state', 'year','month'], ascending=[1,1,1])
result = result[:len(power_agg)]
power_agg = power_agg[:len(result)]


C:\Users\mmorenocabrera\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [6]:
state = result['state']
result = pd.get_dummies(result, columns = ['state'], sparse = True)
result = pd.get_dummies(result, columns = ['month'], sparse = True)
result.head(3)

,year,avg_temp,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,state_DE,state_FL,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
74,2001,1.957143,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2001,15.907143,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2001,16.957143,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from sklearn.linear_model import LinearRegression


result = result.fillna(0)

y = power_agg.values
X = result


#test_red = test[((test.columns & train.columns) & selectColumns).values]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1729)
print ('Division_Set_Shapes:', X.shape, y.shape)
print ('Validation_Set_Shapes:', X_train.shape, X_test.shape)



print ('')

lr = LinearRegression()
lr.fit(X_train, y_train)


('Division_Set_Shapes:', (8975, 62), (8975L,))
('Validation_Set_Shapes:', (6282, 62), (2693, 62))



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
import time
n = len(X_train)

for i in range(n/100):
    i=1
    time.sleep(10)
    predResults = lr.predict(X_test[i*100:((i+1)*100)])
    st = state[i*100:((i+1)*100)]
    res = pd.DataFrame({'state' : state[X_test[i*100:((i+1)*100)].index].values,
                        'preds' : predResults})
    res.to_gbq( 'heatwave.predictions',project_id = 'crested-trilogy-136023',if_exists='append',private_key = key)   




Streaming Insert is 100% Complete





Streaming Insert is 100% Complete




KeyboardInterrupt: 

In [9]:
print state[X_test[i*100:((i+1)*100)].index],predResults

res = pd.DataFrame({'state' : state[X_test[i*10:((i+1)*10)].index].values,
 'preds' : predResults})

2354    IN
8148    VT
2876    LA
6862    RI
516     AZ
56      AL
888     CO
1561    GA
3576    MI
5225    NH
Name: state, dtype: object [  3330.21875  11918.03125   6978.59375   4386.6875    3839.         898.625
  19783.9375    3166.4375    5081.8125    6685.6875 ]


In [10]:
res.to_gbq( 'heatwave.predictions',project_id = 'crested-trilogy-136023',if_exists='replace',private_key = key)




Streaming Insert is 100% Complete




In [12]:
print res

         preds state
0   3330.21875    IN
1  11918.03125    VT
2   6978.59375    LA
3   4386.68750    RI
4   3839.00000    AZ
5    898.62500    AL
6  19783.93750    CO
7   3166.43750    GA
8   5081.81250    MI
9   6685.68750    NH
